# MCC Aggregates Forecasting - Statistical Models

Statistical forecasting methods for MCC aggregates:
- Prophet (5 strategies with CV selection)
- SARIMA (with time limits)


In [ ]:
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

print("Statistical Models for MCC Aggregates Forecasting")
print("=" * 50)


In [ ]:
def detect_environment():
    try:
        import google.colab
        from google.colab import drive
        drive.mount('/content/drive/')
        return 'colab', '/content/drive/MyDrive/fcst'
    except ImportError:
        return 'local', '..'

environment, base_path = detect_environment()
print(f"Environment: {environment}")
print(f"Base path: {base_path}")


In [ ]:
# Statistical forecasting libraries
try:
    from prophet import Prophet
    print("✓ Prophet imported successfully")
except ImportError:
    print("⚠ Prophet not available - install with: pip install prophet")
    Prophet = None

try:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    from statsmodels.tsa.arima.model import ARIMA
    print("✓ SARIMA/ARIMA imported successfully")
except ImportError:
    print("⚠ Statsmodels not available - install with: pip install statsmodels")
    SARIMAX = None
    ARIMA = None

# Import evaluation functions
try:
    import sys
    sys.path.append(base_path)
    from evaluation import evaluate_and_report_mcc
    print("✓ Loaded evaluation functions from project")
except ImportError:
    print("⚠ Could not import evaluation functions - will create simple evaluation")
    
    def evaluate_and_report_mcc(model_name, y_true, y_pred, y_train, categories, print_report=True):
        """Simple evaluation function fallback"""
        from sklearn.metrics import mean_absolute_error, mean_squared_error
        
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        
        # Simple sMAPE calculation
        smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8))
        
        # Simple RMSSE calculation
        naive_error = np.mean(np.abs(np.diff(y_train)))
        rmsse = rmse / (naive_error + 1e-8)
        
        metrics = {
            'MAE': mae,
            'RMSE': rmse,
            'sMAPE_w': smape,
            'RMSSE_w': rmsse
        }
        
        if print_report:
            print(f"\n=== {model_name} Results ===")
            print(f"MAE: {mae:.4f}")
            print(f"RMSE: {rmse:.4f}")
            print(f"sMAPE: {smape:.2f}%")
            print(f"RMSSE: {rmsse:.4f}")
        
        return metrics, f"{model_name} evaluation completed"


In [ ]:
print("Loading baseline statistical dataset...")

# Try parquet first, fallback to CSV
baseline_parquet = f'{base_path}/data/features/baseline_statistical_full.parquet'
baseline_csv = f'{base_path}/data/features/baseline_statistical_full.csv'

try:
    df = pd.read_parquet(baseline_parquet)
    print("✓ Loaded parquet file")
except:
    try:
        df = pd.read_csv(baseline_csv)
        print("✓ Loaded CSV file")
    except:
        print("❌ Could not load baseline statistical files")
        print("Please run feature_engineering_final.py first to create the baseline features")
        exit()

# Convert date column and sort
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['client_id', 'category', 'date'])

print(f"Total rows: {len(df):,}")
print(f"Unique series: {df.groupby(['client_id', 'category']).ngroups:,}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Train records: {(df['split'] == 'train').sum():,}")
print(f"Test records: {(df['split'] == 'test').sum():,}")

# Sample time series with minimum length requirement
np.random.seed(42)
series_stats = df.groupby(['client_id', 'category']).agg({
    'split': 'count',  # total records
    'amount': 'count'  # verify same count
}).reset_index()
series_stats.columns = ['client_id', 'category', 'total_records', 'amount_count']
series_ids = series_stats[series_stats['total_records'] >= 104]  # At least 104 weeks

# Prophet: 1000 series (fast model)
prophet_series = series_ids.sample(n=min(1000, len(series_ids)), random_state=42)
df_prophet = df.merge(prophet_series[['client_id', 'category']], on=['client_id', 'category'])

# SARIMA: 150 series (slow model)
sarima_series = series_ids.sample(n=min(150, len(series_ids)), random_state=123)
df_sarima = df.merge(sarima_series[['client_id', 'category']], on=['client_id', 'category'])

print(f"Prophet sample: {len(prophet_series)} series, {len(df_prophet):,} rows")
print(f"SARIMA sample: {len(sarima_series)} series, {len(df_sarima):,} rows")


In [ ]:
def create_train_test_split_from_split_column(df_input):
    """Use existing split column to create train/test splits"""
    results = []
    
    for (client_id, category), group in df_input.groupby(['client_id', 'category']):
        group = group.sort_values('date')
        train_data = group[group['split'] == 'train'].copy()
        test_data = group[group['split'] == 'test'].copy()
        
        # Only include series with both train and test data
        if len(train_data) > 0 and len(test_data) > 0:
            results.append({
                'client_id': client_id,
                'category': category,
                'train_data': train_data,
                'test_data': test_data,
                'train_size': len(train_data)
            })
    
    return results

prophet_splits = create_train_test_split_from_split_column(df_prophet)
sarima_splits = create_train_test_split_from_split_column(df_sarima)

print(f"Prophet valid splits: {len(prophet_splits)}")
print(f"SARIMA valid splits: {len(sarima_splits)}")


In [ ]:
if Prophet is not None:
    def prophet_strategy_1(train_data):
        """Basic Prophet with weekly seasonality"""
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='multiplicative'
        )
        return model

    def prophet_strategy_2(train_data):
        """Prophet with additional regressors"""
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            seasonality_mode='additive'
        )
        model.add_regressor('month')
        model.add_regressor('quarter')
        return model

    def prophet_strategy_3(train_data):
        """Prophet with custom seasonalities"""
        model = Prophet(
            yearly_seasonality=False,
            weekly_seasonality=False,
            daily_seasonality=False
        )
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        model.add_seasonality(name='quarterly', period=91.25, fourier_order=3)
        return model

    def prophet_strategy_4(train_data):
        """Prophet with trend changepoints"""
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0
        )
        return model

    def prophet_strategy_5(train_data):
        """Prophet with holidays and growth"""
        model = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            growth='linear',
            seasonality_mode='multiplicative',
            changepoint_prior_scale=0.1
        )
        return model

    prophet_strategies = {
        'strategy_1': prophet_strategy_1,
        'strategy_2': prophet_strategy_2,
        'strategy_3': prophet_strategy_3,
        'strategy_4': prophet_strategy_4,
        'strategy_5': prophet_strategy_5
    }


In [ ]:
    def evaluate_prophet_strategy(strategy_name, strategy_func, splits_sample, max_series=100):
        """Evaluate Prophet strategy on small CV dataset"""
        print(f"Evaluating Prophet {strategy_name}...")
        
        errors = []
        successful_fits = 0
        start_total_time = time.time()
        
        # Use small sample for CV
        cv_splits = splits_sample[:max_series]
        
        for split in cv_splits:
            # Check total time limit
            if time.time() - start_total_time > 60:
                print(f"  {strategy_name}: Timeout reached after {successful_fits} series")
                break
                
            try:
                train_data = split['train_data'].copy()
                test_data = split['test_data'].copy()
                
                # Prepare Prophet format
                prophet_train = pd.DataFrame({
                    'ds': train_data['date'],
                    'y': train_data['amount']
                })
                
                # Add regressors if needed
                if strategy_name == 'strategy_2':
                    prophet_train['month'] = train_data['month']
                    prophet_train['quarter'] = train_data['quarter']
                
                # Fit model
                model = strategy_func(train_data)
                model.fit(prophet_train)
                
                # Make forecast
                future = model.make_future_dataframe(periods=len(test_data), freq='W')
                if strategy_name == 'strategy_2':
                    future['month'] = pd.to_datetime(future['ds']).dt.month
                    future['quarter'] = pd.to_datetime(future['ds']).dt.quarter
                
                forecast = model.predict(future)
                predictions = forecast['yhat'].iloc[-len(test_data):].values
                
                # Calculate error using consistent sMAPE formula
                actual = test_data['amount'].values
                # Use same sMAPE calculation as in evaluation.py
                smape_error = 100 * np.mean(2 * np.abs(predictions - actual) / (np.abs(actual) + np.abs(predictions) + 1e-8))
                errors.append(smape_error)
                successful_fits += 1
                
            except Exception as e:
                continue
        
        avg_error = np.mean(errors) if errors else float('inf')
        elapsed_time = time.time() - start_total_time
        print(f"{strategy_name}: {successful_fits}/{len(cv_splits)} successful fits, Avg sMAPE: {avg_error:.2f}%, Time: {elapsed_time:.1f}s")
        
        return avg_error, successful_fits

    # Run Prophet CV
    print("Running Prophet Cross-Validation...")
    prophet_cv_results = {}

    for strategy_name, strategy_func in prophet_strategies.items():
        error, fits = evaluate_prophet_strategy(strategy_name, strategy_func, prophet_splits)
        prophet_cv_results[strategy_name] = {'error': error, 'fits': fits}

    # Select best Prophet strategy
    best_prophet = min(prophet_cv_results.keys(), key=lambda x: prophet_cv_results[x]['error'])
    print(f"\nBest Prophet strategy: {best_prophet} (sMAPE: {prophet_cv_results[best_prophet]['error']:.2f}%)")


In [ ]:
    def fit_prophet_model(train_data, strategy_name):
        """Fit Prophet model with selected strategy"""
        prophet_train = pd.DataFrame({
            'ds': train_data['date'],
            'y': train_data['amount']
        })
        
        if strategy_name == 'strategy_2':
            prophet_train['month'] = train_data['month']
            prophet_train['quarter'] = train_data['quarter']
        
        model = prophet_strategies[strategy_name](train_data)
        model.fit(prophet_train)
        return model

    print(f"\nRunning Prophet final evaluation with {best_prophet}...")
    prophet_results = []
    prophet_predictions = []
    prophet_actuals = []
    prophet_categories = []

    # Use 500 series for final evaluation
    eval_splits = prophet_splits[:500]
    start_prophet_time = time.time()

    for i, split in enumerate(eval_splits):
        # Check total time limit
        if time.time() - start_prophet_time > 60:
            print(f"Prophet: Timeout reached after {i} series")
            break
            
        try:
            train_data = split['train_data']
            test_data = split['test_data']
            
            # Fit model
            model = fit_prophet_model(train_data, best_prophet)
            
            # Forecast
            future = model.make_future_dataframe(periods=len(test_data), freq='W')
            if best_prophet == 'strategy_2':
                future['month'] = pd.to_datetime(future['ds']).dt.month
                future['quarter'] = pd.to_datetime(future['ds']).dt.quarter
            
            forecast = model.predict(future)
            predictions = forecast['yhat'].iloc[-len(test_data):].values
            
            # Store results
            actual = test_data['amount'].values
            prophet_predictions.extend(predictions)
            prophet_actuals.extend(actual)
            prophet_categories.extend([split['category']] * len(actual))
            
            if (i + 1) % 50 == 0:
                elapsed = time.time() - start_prophet_time
                print(f"Prophet: Processed {i+1}/{len(eval_splits)} series ({elapsed:.1f}s)")
                
        except Exception as e:
            print(f"Prophet error on series {i+1}: {str(e)[:50]}")
            continue

    prophet_total_time = time.time() - start_prophet_time
    print(f"Prophet completed: {len(prophet_predictions)} predictions in {prophet_total_time:.1f}s")

else:
    print("Prophet not available - skipping Prophet evaluation")
    prophet_predictions = []
    prophet_actuals = []
    prophet_categories = []
    prophet_total_time = 0


In [ ]:
if SARIMAX is not None and ARIMA is not None:
    def fit_sarima_model(train_data):
        """Fit SARIMA model"""
        y = train_data['amount'].values
        
        # Simple SARIMA configuration
        try:
            # Try SARIMA(1,1,1)(1,1,1,52) first
            model = SARIMAX(y, order=(1,1,1), seasonal_order=(1,1,1,52))
            fitted_model = model.fit(disp=False, maxiter=50)
            return fitted_model
            
        except:
            try:
                # Fallback to simpler ARIMA
                model = ARIMA(y, order=(1,1,1))
                fitted_model = model.fit()
                return fitted_model
            except:
                return None

    print("Running SARIMA evaluation...")
    sarima_predictions = []
    sarima_actuals = []
    sarima_categories = []
    sarima_timeouts = 0
    start_sarima_time = time.time()

    for i, split in enumerate(sarima_splits):
        # Check total time limit
        if time.time() - start_sarima_time > 60:
            print(f"SARIMA: Timeout reached after {i} series")
            break
            
        try:
            train_data = split['train_data']
            test_data = split['test_data']
            
            # Fit SARIMA
            model = fit_sarima_model(train_data)
            
            if model is None:
                sarima_timeouts += 1
                print(f"SARIMA error on series {i+1}")
                continue
            
            # Forecast
            forecast = model.forecast(steps=len(test_data))
            predictions = forecast if hasattr(forecast, '__len__') else [forecast] * len(test_data)
            
            # Store results
            actual = test_data['amount'].values
            sarima_predictions.extend(predictions[:len(actual)])
            sarima_actuals.extend(actual)
            sarima_categories.extend([split['category']] * len(actual))
            
            if (i + 1) % 25 == 0:
                elapsed = time.time() - start_sarima_time
                print(f"SARIMA: Processed {i+1}/{len(sarima_splits)} series ({elapsed:.1f}s)")
                
        except Exception as e:
            print(f"SARIMA error on series {i+1}: {str(e)[:50]}")
            continue

    sarima_total_time = time.time() - start_sarima_time
    print(f"SARIMA completed: {len(sarima_predictions)} predictions, {sarima_timeouts} errors in {sarima_total_time:.1f}s")

    # Mark as inappropriate if too slow
    if sarima_total_time > 60:
        print("WARNING: SARIMA marked as inappropriate due to slow performance")

else:
    print("SARIMA/ARIMA not available - skipping SARIMA evaluation")
    sarima_predictions = []
    sarima_actuals = []
    sarima_categories = []
    sarima_total_time = 0
    sarima_timeouts = 0


In [ ]:
results = {}

# Prophet evaluation
if prophet_predictions and prophet_total_time <= 60:
    # Correctly gather training data only from series that were actually evaluated
    prophet_train_data = []
    series_count = 0
    for i, split in enumerate(eval_splits):
        if series_count * 4 >= len(prophet_predictions):
            break
        prophet_train_data.append(split['train_data']['amount'].values)
        series_count += 1
    
    prophet_train_data = np.concatenate(prophet_train_data)
    
    prophet_metrics, prophet_report = evaluate_and_report_mcc(
        f"Prophet ({best_prophet})",
        np.array(prophet_actuals),
        np.array(prophet_predictions),
        prophet_train_data,
        np.array(prophet_categories),
        print_report=True
    )
    results['prophet'] = prophet_metrics
elif prophet_total_time > 60:
    print("Prophet excluded from metrics due to timeout (>60s)")

# SARIMA evaluation
if sarima_predictions and sarima_total_time <= 60:
    # Correctly gather training data only from series that were actually evaluated
    sarima_train_data = []
    series_count = 0
    for i, split in enumerate(sarima_splits):
        if series_count * 4 >= len(sarima_predictions):
            break
        sarima_train_data.append(split['train_data']['amount'].values)
        series_count += 1
    
    sarima_train_data = np.concatenate(sarima_train_data)
    
    sarima_metrics, sarima_report = evaluate_and_report_mcc(
        "SARIMA",
        np.array(sarima_actuals),
        np.array(sarima_predictions),
        sarima_train_data,
        np.array(sarima_categories),
        print_report=True
    )
    results['sarima'] = sarima_metrics
elif sarima_total_time > 60:
    print("SARIMA excluded from metrics due to timeout (>60s)")


In [ ]:
print("\nSaving results...")

# Prophet strategy details
prophet_strategy_details = {
    'strategy_1': 'Basic Prophet with weekly/yearly seasonality (multiplicative)',
    'strategy_2': 'Prophet with additional regressors (month, quarter) (additive)',
    'strategy_3': 'Prophet with custom seasonalities (monthly, quarterly)',
    'strategy_4': 'Prophet with trend changepoints (changepoint_prior_scale=0.05)',
    'strategy_5': 'Prophet with holidays and growth (multiplicative, changepoint_prior_scale=0.1)'
}

# Add model info
results['model_info'] = {
    'environment': environment,
    'prophet_available': Prophet is not None,
    'sarima_available': SARIMAX is not None and ARIMA is not None,
    'evaluation_date': datetime.now().isoformat()
}

if Prophet is not None:
    results['model_info'].update({
        'prophet_strategy': best_prophet,
        'prophet_strategy_description': prophet_strategy_details[best_prophet],
        'prophet_cv_results': prophet_cv_results,
        'prophet_series_evaluated': len([s for s in eval_splits if len(prophet_predictions) > 0]),
        'prophet_predictions': len(prophet_predictions),
        'prophet_time': prophet_total_time,
        'prophet_appropriate': prophet_total_time <= 60,
    })

if SARIMAX is not None and ARIMA is not None:
    results['model_info'].update({
        'sarima_series_evaluated': len([s for s in sarima_splits if len(sarima_predictions) > 0]),
        'sarima_predictions': len(sarima_predictions),
        'sarima_time': sarima_total_time,
        'sarima_timeouts': sarima_timeouts,
        'sarima_appropriate': sarima_total_time <= 60,
    })

# Save to JSON
output_file = f'{base_path}/MCC Aggregates Forecasting/Statistical models/statistical_models_results.json'
with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Results saved to {output_file}")


In [ ]:
print("\n" + "="*60)
print("STATISTICAL MODELS SUMMARY")
print("="*60)

if Prophet is not None:
    print(f"Prophet Strategy Selection:")
    print(f"Winner: {best_prophet} - {prophet_strategy_details[best_prophet]}")
    print(f"CV Results:")
    for strategy, result in prophet_cv_results.items():
        print(f"  {strategy}: sMAPE {result['error']:.2f}%, Fits: {result['fits']}")

    if 'prophet' in results:
        print(f"\nProphet ({best_prophet}) - APPROPRIATE:")
        print(f"  - Series evaluated: {results['model_info']['prophet_series_evaluated']}")
        print(f"  - Predictions made: {len(prophet_predictions)}")
        print(f"  - Total time: {prophet_total_time:.1f}s")
        print(f"  - sMAPE: {results['prophet']['sMAPE_w']:.2f}%")
        print(f"  - RMSSE: {results['prophet']['RMSSE_w']:.4f}")
    elif prophet_total_time > 60:
        print(f"\nProphet ({best_prophet}) - INAPPROPRIATE (>{prophet_total_time:.1f}s)")
else:
    print("Prophet not available")

if SARIMAX is not None and ARIMA is not None:
    if 'sarima' in results:
        print(f"\nSARIMA - APPROPRIATE:")
        print(f"  - Series evaluated: {results['model_info']['sarima_series_evaluated']}")
        print(f"  - Predictions made: {len(sarima_predictions)}")
        print(f"  - Total time: {sarima_total_time:.1f}s")
        print(f"  - Errors: {sarima_timeouts}")
        print(f"  - sMAPE: {results['sarima']['sMAPE_w']:.2f}%")
        print(f"  - RMSSE: {results['sarima']['RMSSE_w']:.4f}")
    elif sarima_total_time > 60:
        print(f"\nSARIMA - INAPPROPRIATE (>{sarima_total_time:.1f}s)")
else:
    print("SARIMA not available")

print(f"\nEnvironment: {environment}")
print("Statistical models evaluation completed!") 